# Import All Libaries

In [11]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded
from tira.rest_api_client import Client
ensure_pyterrier_is_loaded()
import pyterrier as pt
#import pandas as pd

tira = Client()

# Load the Dataset and the Index


In [12]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
dataset = 'ir-acl-anthology-20240504-training'
pt_dataset = pt.get_dataset(f'irds:ir-benchmarks/{dataset}')
#bm25 = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 Re-Rank (tira-ir-starter-pyterrier)', dataset)

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

In [13]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

## Stopwords

In [14]:
def create_index(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

chatGPTStopwords =[
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 
    'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
    'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
    'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 
    'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
    'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
    'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 
    'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 
    'will', 'just', 'don', 'should', 'now'
]

ntlkStopwords = [
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", 
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", 
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", 
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that", 
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", 
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", 
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", 
    "at", "by", "for", "with", "about", "against", "between", "into", "through", 
    "during", "before", "after", "above", "below", "to", "from", "up", "down", 
    "in", "out", "on", "off", "over", "under", "again", "further", "then", 
    "once", "here", "there", "when", "where", "why", "how", "all", "any", 
    "both", "each", "few", "more", "most", "other", "some", "such", "no", 
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", 
    "t", "can", "will", "just", "don", "should", "now", "d", "ll", "m", "o", 
    "re", "ve", "y", "ain", "aren", "couldn", "didn", "doesn", "hadn", "hasn", 
    "haven", "isn", "ma", "mightn", "mustn", "needn", "shan", "shouldn", 
    "wasn", "weren", "won", "wouldn"
]

spacyStopwords = [
    "'d", "'ll", "'m", "'re", "'s", "'ve", "a", "about", "above", "across", "after", 
    "afterwards", "again", "against", "all", "almost", "alone", "along", "already", 
    "also", "although", "always", "am", "among", "amongst", "amount", "an", "and", 
    "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", 
    "around", "as", "at", "back", "be", "became", "because", "become", "becomes", 
    "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", 
    "besides", "between", "beyond", "both", "bottom", "but", "by", "ca", "call", 
    "can", "cannot", "could", "did", "do", "does", "doing", "done", "down", "due", 
    "during", "each", "eight", "either", "eleven", "else", "elsewhere", "empty", 
    "enough", "even", "ever", "every", "everyone", "everything", "everywhere", 
    "except", "few", "fifteen", "fifty", "first", "five", "for", "former", "formerly", 
    "forty", "four", "from", "front", "full", "further", "get", "give", "go", "had", 
    "has", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", 
    "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", 
    "hundred", "i", "if", "in", "indeed", "into", "is", "it", "its", "itself", "just", 
    "keep", "last", "latter", "latterly", "least", "less", "made", "make", "many", 
    "may", "me", "meanwhile", "might", "mine", "more", "moreover", "most", "mostly", 
    "move", "much", "must", "my", "myself", "n't", "name", "namely", "neither", 
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", 
    "not", "nothing", "now", "nowhere", "n‘t", "n’t", "of", "off", "often", "on", 
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", 
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", 
    "put", "quite", "rather", "re", "really", "regarding", "same", "say", "see", 
    "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", 
    "show", "side", "since", "six", "sixty", "so", "some", "somehow", "someone", 
    "something", "sometime", "sometimes", "somewhere", "still", "such", "take", 
    "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", 
    "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", 
    "they", "third", "this", "those", "though", "three", "through", "throughout", 
    "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", 
    "twenty", "two", "under", "unless", "until", "up", "upon", "us", "used", "using", 
    "various", "very", "via", "was", "we", "well", "were", "what", "whatever", 
    "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", 
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", 
    "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", 
    "‘d", "‘ll", "‘m", "‘re", "‘s", "‘ve", "’d", "’ll", "’m", "’re", "’s", "’ve"
]

index = create_index(pt_dataset.get_corpus_iter(), chatGPTStopwords)
bm25_chatGPTStopwords = pt.BatchRetrieve(index, wmodel="BM25")

index2 = create_index(pt_dataset.get_corpus_iter(),ntlkStopwords)
bm25_ntlkStopwords = pt.BatchRetrieve(index2, wmodel="BM25")

index3 = create_index(pt_dataset.get_corpus_iter(),spacyStopwords)
bm25_spacyStopwords = pt.BatchRetrieve(index3, wmodel="BM25")

ir-benchmarks/ir-acl-anthology-20240504-training documents:  71%|███████   | 90143/126958 [00:17<00:05, 7027.87it/s]

15:37:28.571 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-benchmarks/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:23<00:00, 5487.95it/s] 


15:37:36.063 [ForkJoinPool-4-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-benchmarks/ir-acl-anthology-20240504-training documents:  70%|███████   | 89143/126958 [00:16<00:07, 4920.74it/s]

...

15:38:02.529 [ForkJoinPool-5-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


ir-benchmarks/ir-acl-anthology-20240504-training documents:  65%|██████▌   | 82817/126958 [00:16<00:13, 3220.92it/s]

15:38:20.919 [ForkJoinPool-6-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1980.sigirconf_conference-80.24) - further warnings are suppressed


ir-benchmarks/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:22<00:00, 5582.00it/s] 


15:38:29.095 [ForkJoinPool-6-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 4 empty documents


## Query Expansion with Large Language Models

In [15]:
gpt_sq_zs = tira.pt.transform_queries('ir-benchmarks/tu-dresden-03/qe-gpt3.5-sq-zs', dataset, prefix='llm_expansion_')


In [16]:
tokeniser = pt.autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

def pt_tokenize(text):
    return ' '.join(tokeniser.getTokens(text))

def expand_query(topic):
  ret = ' '.join([topic['query'], topic['query'], topic['query'],  topic['query'],  topic['query'], topic['llm_expansion_query']])

  # apply the tokenization
  return pt_tokenize(ret)

# we wrap this into an pyterrier transformer
# Documentation: https://pyterrier.readthedocs.io/en/latest/apply.html
pt_expand_query = pt.apply.query(expand_query)

In [17]:
pipeline_gpt_sq_zs = (gpt_sq_zs >> pt_expand_query) >> bm25
pipeline_gpt_sq_zs_chatgptstopwords = (gpt_sq_zs >> pt_expand_query) >> bm25_chatGPTStopwords
pipeline_gpt_sq_zs_ntlkstopwords = (gpt_sq_zs >> pt_expand_query) >> bm25_ntlkStopwords
pipeline_gpt_sq_zs_spacystopwords = (gpt_sq_zs >> pt_expand_query) >> bm25_spacyStopwords

### Evaluation

In [18]:
pt.Experiment(
    retr_systems=[pipeline_gpt_sq_zs, pipeline_gpt_sq_zs_chatgptstopwords, pipeline_gpt_sq_zs_ntlkstopwords, pipeline_gpt_sq_zs_spacystopwords],
    topics=pt_dataset.get_topics('text'),
    qrels=pt_dataset.get_qrels(),
    names=['GPT-SQ-ZS', 'GPT-SQ-ZS+GPT-Stopwords','GPT-SQ-ZS+NLTK-Stopwords','GPT-SQ-ZS+SpaCy-Stopwords'],
    eval_metrics=['recall_1000', 'ndcg_cut.10', 'recip_rank']
)

,name,recall_1000,ndcg_cut.10,recip_rank
0,GPT-SQ-ZS,0.829955,0.364157,0.644563
1,GPT-SQ-ZS+GPT-Stopwords,0.834277,0.369355,0.644707
2,GPT-SQ-ZS+NLTK-Stopwords,0.835327,0.368129,0.644722
3,GPT-SQ-ZS+SpaCy-Stopwords,0.835736,0.369013,0.634675
